<a href="https://colab.research.google.com/github/VellummyilumVinoth/Train_And_Test_Using_ALBERT/blob/main/FinetunedAlbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [3]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.7 MB/s eta 0:00:00


In [5]:
from transformers import AlbertForMaskedLM,RobertaTokenizerFast

# Load the CodeBERTa tokenizer and ALBERT model
tokenizer = RobertaTokenizerFast.from_pretrained('huggingface/CodeBERTa-small-v1')

model = AlbertForMaskedLM.from_pretrained('albert-base-v2')

In [6]:
import csv

variable_names = []
statements = []

with open('output.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        # Check if the row contains at least two columns
        if len(row) >= 2:
            # Append the variable name and source statement to their respective lists
            variable_names.append(row[0].lower())
            statements.append(row[1].replace(';', '').lower())  # Remove the ';' symbol

In [7]:
variable_names

['intarray',
 'persons',
 'personlist',
 'personarray',
 'intarray',
 'numbers',
 'numbersarray',
 'numberslist',
 'count',
 'name',
 'age',
 'isauthorized',
 'israining',
 'isflag',
 'salary',
 'names',
 'payload',
 'datelist',
 'datearray',
 'totalprice',
 'message',
 'isfound',
 'employees',
 'students',
 'order',
 'student',
 'person',
 'persons',
 'persons',
 'num',
 'daysofweek',
 'utcoffset',
 'civiltimesecfield',
 'utcoffsetsecfield',
 'civiltimeseconds',
 'utcoffsetseconds',
 'utcoffset',
 'civiltimesecfield',
 'utcoffsethours',
 'utcoffsetminutes',
 'utcoffsetseconds',
 'utcoffset',
 'timeabbrevfield',
 'timeabbrev',
 'jsonkey',
 'node',
 'node',
 'arrayentrytagkey',
 'item',
 'jmap',
 'attributeprefix',
 'element',
 'index',
 'prefix',
 'elementname',
 'namespaceurl',
 'newattributes',
 'value',
 'attributes',
 'attr',
 'index',
 'suffix',
 'startindex',
 'prefix',
 'namespaceurl',
 'startindex',
 'location',
 'namespaces',
 'prefix',
 'cn',
 'value',
 'c0',
 'c1',
 'size',


In [8]:
statements

['int[] intarray = getnumbers()',
 'person[] persons = getpersons()',
 'person[] personlist = getpersons()',
 'int[] personarray = getpersons()',
 'int[] intlist = getnumbers()',
 'int[] numbers = getnumbers()',
 'int[] numbersarray = getnumbers()',
 'int[] numberslist = getnumbers()',
 'int count = getcount()',
 'string name = getname()',
 'int age = getage()',
 'boolean isauthorized = checkauthorization()',
 'boolean israining = false',
 'boolean isflag = true',
 'float salary = getsalary()',
 'string[] names = getnames()',
 'json payload = getpayload()',
 'date[] datelist = []',
 'date[] datearray = []',
 'float totalprice = calculatetotalprice()',
 'string message = getmessage()',
 'boolean isfound = finditem()',
 'employee[] employees = getemployees()',
 'students[] students = getstudents()',
 'order order = getorder()',
 'student student',
 "person person = {name: 'john'}",
 'person[] persons = getpersons()',
 "person[] persons = getpersons(10, 'sales')",
 'int num',
 'dayofweek[

In [9]:
import re

def clean_text(line):
    line = re.sub(r'-+',' ',line)
    line = re.sub(r'[^a-zA-Z, ]+'," ",line)
    line = re.sub(r'[ ]+'," " ,line)
    line += ""
    return line

source_statements = []
len_lst = []
for line in statements:
    if len(line.split(" ")) >=0:
        line = clean_text(line)
        source_statements.append(line)
        len_lst.append(len(line.split(" ")))


In [10]:
source_statements

['int intarray getnumbers ',
 'person persons getpersons ',
 'person personlist getpersons ',
 'int personarray getpersons ',
 'int intlist getnumbers ',
 'int numbers getnumbers ',
 'int numbersarray getnumbers ',
 'int numberslist getnumbers ',
 'int count getcount ',
 'string name getname ',
 'int age getage ',
 'boolean isauthorized checkauthorization ',
 'boolean israining false',
 'boolean isflag true',
 'float salary getsalary ',
 'string names getnames ',
 'json payload getpayload ',
 'date datelist ',
 'date datearray ',
 'float totalprice calculatetotalprice ',
 'string message getmessage ',
 'boolean isfound finditem ',
 'employee employees getemployees ',
 'students students getstudents ',
 'order order getorder ',
 'student student',
 'person person name john ',
 'person persons getpersons ',
 'person persons getpersons , sales ',
 'int num',
 'dayofweek daysofweek sunday, monday, tuesday, wednesday, thursday, friday, saturday ',
 'zoneoffset utcoffset zoneoffset civiltime

In [11]:
# from huggingface_hub import add_space_secret
import pandas as pd
import torch
import os
import re
from transformers import AdamW, Trainer, TrainingArguments, DataCollatorForLanguageModeling

import random

class VariableNamesDataset(torch.utils.data.Dataset):
    def __init__(self, variable_names, source_statements, tokenizer, mask_probability):
        self.variable_names = variable_names
        self.source_statements = source_statements
        self.tokenizer = tokenizer
        self.mask_probability = mask_probability

        # Add variable name tokens to the tokenizer
        # self.tokenizer.add_tokens(variable_names)

    def __len__(self):
        return len(self.source_statements)
    
    def __getitem__(self, idx):
        variable_name = str(self.variable_names[idx])
        source_statement = str(self.source_statements[idx])

        print("variable names: ",len(variable_name))
        print("source statements: ",len(source_statement))

        # Tokenize the variable name and source statement
        variable_name_tokens = self.tokenizer.tokenize(variable_name)
        source_statement_tokens = self.tokenizer.tokenize(source_statement)

        variable_name_tokens = [token.replace('Ġ', '').lower() for token in variable_name_tokens]
        source_statement_tokens = [token.replace('Ġ', '').lower() for token in source_statement_tokens]

        print("Variable name tokens:", variable_name_tokens)
        print("Source statement tokens:", source_statement_tokens)

        # Select a variable name tokens to mask
        variable_name_indices = [i for i, token in enumerate(source_statement_tokens) if token in variable_name_tokens]

        print("Variable name indices:", variable_name_indices)

        num_to_mask = int(len(variable_name_indices)* self.mask_probability)
        indices_to_mask = random.sample(variable_name_indices, num_to_mask)

        print("Num to mask:", num_to_mask)
        print("Indices to mask:", indices_to_mask)

        # Replace the selected variable name tokens with the [MASK] token
        for i in indices_to_mask:
            source_statement_tokens[i] = '<mask>'

        masked_source_statement = ' '.join(source_statement_tokens)
        print("Masked source statement:", masked_source_statement)

        # Tokenize the masked source statement
        input_ids = self.tokenizer.encode(
            masked_source_statement,
            add_special_tokens=False,
            truncation=True,
            padding='max_length',
            max_length=128
        )

        print("Input IDs:", input_ids)

        return torch.tensor(input_ids)
 

In [12]:
# Split the data into train and test sets
train_size = int(len(source_statements) * 0.9)
train_variable_names = variable_names[:train_size]
train_source_statements = source_statements[:train_size]
test_variable_names = variable_names[train_size:]
test_source_statements = source_statements[train_size:]

In [13]:
# Prepare the data for training and testing
train_dataset = VariableNamesDataset(train_variable_names, train_source_statements, tokenizer, mask_probability = 1)

test_dataset = VariableNamesDataset(test_variable_names, test_source_statements, tokenizer, mask_probability = 0)

In [14]:
train_dataset[59]

variable names:  4
source statements:  37
Variable name tokens: ['attr']
Source statement tokens: ['map', 'json', 'error', 'attr', 'j', 'tree', 'ensure', 'type', '']
Variable name indices: [3]
Num to mask: 1
Indices to mask: [3]
Masked source statement: map json error <mask> j tree ensure type 
Input IDs: [1025, 1644, 670, 4, 719, 3456, 4281, 882, 225, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


tensor([1025, 1644,  670,    4,  719, 3456, 4281,  882,  225,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1])

In [15]:
test_dataset[25]

variable names:  4
source statements:  64
Variable name tokens: ['temp']
Source statement tokens: ['handle', 'temp', 'replace', 'string', 'java', 'from', 'string', 'msg', ',', 'java', 'from', 'string', '']
Variable name indices: [1]
Num to mask: 0
Indices to mask: []
Masked source statement: handle temp replace string java from string msg , java from string 
Input IDs: [2128, 2779, 2886, 679, 3671, 772, 679, 1867, 3161, 3671, 772, 679, 225, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


tensor([2128, 2779, 2886,  679, 3671,  772,  679, 1867, 3161, 3671,  772,  679,
         225,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1])

In [16]:
import torch
torch.cuda.empty_cache()

In [17]:
# Prepare the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

model.resize_token_embeddings(len(tokenizer))

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "steps",   # Evaluation and Save happens every X steps
    eval_steps = 50,                 # Evaluation happens every X steps
    save_steps = 1000,               # Save checkpoint every X steps
    num_train_epochs = 15,           # Total number of training epochs
    learning_rate = 2e-5,            # Learning rate
    per_device_train_batch_size=32,  # Batch size per device during training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps = 500,              # Number of warmup steps
    weight_decay = 0.01,             # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=1000,              # Log every X steps
)

# Instantiate the Trainer class and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


variable names:  4
source statements:  27
Variable name tokens: ['body']
Source statement tokens: ['string', 'body', 'metadata', 'ref', ',', '']
Variable name indices: [1]
Num to mask: 1
Indices to mask: [1]
Masked source statement: string <mask> metadata ref , 
Input IDs: [492, 4, 2958, 1872, 3161, 225, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
variable names:  7
source statements:  83
Variable name tokens: ['payload']
Source statement tokens: ['var', 'payload', 'check', 'self', 'grpc', 'client', 'executes', 'impl', 'er', 'pc', 'repeated', 'types', 'service', 'struct', 'call', '']
Variable name indices: [1]
Num to mask: 1
Indices to mask: [1]
Masked source statement:

Step,Training Loss,Validation Loss
50,No log,12.399999
100,No log,10.013046
150,No log,9.320601
200,No log,9.329863
250,No log,9.284191
300,No log,8.385802
350,No log,7.962529
400,No log,7.985116
450,No log,7.620856
500,No log,7.023736


Streaming output truncated to the last 5000 lines.
Variable name indices: [1, 2, 3, 27, 35]
Num to mask: 5
Indices to mask: [35, 3, 27, 2, 1]
Masked source statement: handle <mask> <mask> <mask> ll v md ib u ilder creates ub rout in etype self l lv md ib u ilder , fileobj , parameter arr <mask> object , paramet ertypes length , flag <mask> 
Input IDs: [2128, 4, 4, 4, 14785, 438, 4738, 18397, 1172, 225, 1009, 10777, 16581, 5522, 333, 36051, 407, 451, 7728, 4738, 18397, 1172, 225, 1009, 3161, 24360, 3161, 2186, 4651, 4, 976, 3161, 50993, 225, 40340, 1821, 3161, 3916, 4, 225, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
variable names:  9
source statements:  44
Variable name tokens: ['remainder']
Source statement tokens: ['string', 'remainder', 'clean', 'base', 'substring',

TrainOutput(global_step=1125, training_loss=5.951056477864583, metrics={'train_runtime': 1154.2215, 'train_samples_per_second': 30.995, 'train_steps_per_second': 0.975, 'total_flos': 201609940377600.0, 'train_loss': 5.951056477864583, 'epoch': 15.0})

In [18]:
import os

output_dir = os.path.expanduser('/content/finetuned_albert')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('/content/finetuned_albert/tokenizer_config.json',
 '/content/finetuned_albert/special_tokens_map.json',
 '/content/finetuned_albert/vocab.json',
 '/content/finetuned_albert/merges.txt',
 '/content/finetuned_albert/added_tokens.json',
 '/content/finetuned_albert/tokenizer.json')

In [19]:
import torch
from transformers import AlbertForMaskedLM,RobertaTokenizerFast
from tabulate import tabulate

# Load the trained model and tokenizer
output_dir = os.path.expanduser("/content/finetuned_albert")
model = AlbertForMaskedLM.from_pretrained(output_dir)
tokenizer = RobertaTokenizerFast.from_pretrained(output_dir)

# Define a sample masked statement
masked_statement = "int <mask> = getCount();"

# Tokenize the masked statement
input_ids = tokenizer.encode(masked_statement, add_special_tokens=False, return_tensors='pt')

# Find the position of the masked token
masked_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1][0].item()

# Generate predictions for the masked token using the fine-tuned model
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Get the top 5 predictions and their probability scores from the fine-tuned model
probs_ft = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
top_k_ft = torch.topk(probs_ft, k=5)

# Create a table with the top predictions and their probabilities from both models
table = [["Fine-Tuned Model", f"{tokenizer.mask_token}"] + [tokenizer.convert_ids_to_tokens([idx])[0].replace('Ġ', '').lower() for idx in top_k_ft.indices],
         ["Probability", ""] + [f"{probs_ft[idx].item():.4f}" for idx in top_k_ft.indices]]

# Print the table
print(tabulate(table, headers="firstrow", tablefmt="fancy_grid"))


╒════════════════════╤══════════╤═════════╤════════╤════════╤════════╤════════╕
│ Fine-Tuned Model   │ <mask>   │   check │    int │      s │     ir │   type │
╞════════════════════╪══════════╪═════════╪════════╪════════╪════════╪════════╡
│ Probability        │          │  0.2108 │ 0.0532 │ 0.0512 │ 0.0228 │ 0.0191 │
╘════════════════════╧══════════╧═════════╧════════╧════════╧════════╧════════╛


In [20]:
import torch
from transformers import AlbertForMaskedLM,RobertaTokenizer

# Load the trained model and tokenizer
output_dir = os.path.expanduser("/content/finetuned_albert")
finetuned_model = AlbertForMaskedLM.from_pretrained(output_dir)
finetuned_tokenizer = RobertaTokenizer.from_pretrained(output_dir)

# Load the ALBERT model and tokenizer
base_model = AlbertForMaskedLM.from_pretrained('albert-base-v2')
base_tokenizer = RobertaTokenizer.from_pretrained('huggingface/CodeBERTa-small-v1')

base_model.resize_token_embeddings(len(base_tokenizer))

# Define a list of sample masked statements
masked_statements = ["int <mask> = getCount();", "Student <mask>;", "Person <mask> = {name: 'John'};", "int[] <mask> = getNumbers();", "Person[] <mask> = getPersons();", "Person[] <mask> = getPersons(10, 'Sales');"]

# Loop through each masked statement and generate predictions for both models
for masked_statement in masked_statements:
    print(f"Masked statement: {masked_statement}\n")
    # Tokenize the masked statement for the fine-tuned model
    input_ids = finetuned_tokenizer.encode(masked_statement, add_special_tokens=False, return_tensors='pt')
    masked_token_index = torch.where(input_ids == finetuned_tokenizer.mask_token_id)[1][0].item()
    with torch.no_grad():
        outputs = finetuned_model(input_ids)
        predictions = outputs[0]
    probs = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
    top_k = torch.topk(probs, k=5)
    # Print the top 5 predictions and their probability scores for the fine-tuned model
    print("Fine-tuned model predictions:")
    print("{:<20} {:<20}".format('Prediction', 'Probability'))
    for i, idx in enumerate(top_k.indices):
        token = finetuned_tokenizer.convert_ids_to_tokens([idx])[0].replace('Ġ', '').lower()
        prob = top_k.values[i].item()
        print("{:<20} {:.4f}".format(token, prob))
    print("\n")
    
    # Tokenize the masked statement for the base model
    input_ids = base_tokenizer.encode(masked_statement, add_special_tokens=False, return_tensors='pt')
    masked_token_index = torch.where(input_ids == base_tokenizer.mask_token_id)[1][0].item()
    with torch.no_grad():
        outputs = base_model(input_ids)
        predictions = outputs[0]
    probs = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
    top_k = torch.topk(probs, k=5)
    # Print the top 5 predictions and their probability scores for the base model
    print("Base model predictions:")
    print("{:<20} {:<20}".format('Prediction', 'Probability'))
    for i, idx in enumerate(top_k.indices):
        token = base_tokenizer.convert_ids_to_tokens([idx])[0].replace('Ġ', '').lower()
        prob = top_k.values[i].item()
        print("{:<20} {:.4f}".format(token, prob))
    print("\n")


Masked statement: int <mask> = getCount();

Fine-tuned model predictions:
Prediction           Probability         
check                0.2108
int                  0.0532
s                    0.0512
ir                   0.0228
type                 0.0191


Base model predictions:
Prediction           Probability         
serializationexception 0.0821
motifs               0.0728
§                    0.0651
(?:[                 0.0479
=                    0.0443


Masked statement: Student <mask>;

Fine-tuned model predictions:
Prediction           Probability         
cx                   0.0506
h                    0.0485
builder              0.0405
i                    0.0392
some                 0.0327


Base model predictions:
Prediction           Probability         
serializationexception 0.0822
motifs               0.0700
§                    0.0635
(?:[                 0.0487
=                    0.0456


Masked statement: Person <mask> = {name: 'John'};

Fine-tuned model predi